<a href="https://colab.research.google.com/github/marianatiele/Analise-Petrobras/blob/main/LSTM_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
dados = pd.read_excel('/content/Base.xlsx')
dados.shape
dados.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
# dados.head()

In [ ]:
# dados.drop(columns='Data').describe()

In [ ]:
# dados.head()

In [ ]:
# dados.count() / 2
# dados[1248:]

In [ ]:
# dados.tail()

In [ ]:
# dados['PETR4']

In [ ]:
# dados.set_index('Data', inplace = True)

In [ ]:
# dados.set_index('Data')
# fig = go.Figure(data=[go.Scatter(x=dados.index, y=dados['Investimento'], name='Investimento', mode='lines', line=dict(color='black'))])

# fig.update_layout(title='Investimento na PETR4 de 2014 á 2024',
#                   xaxis_title='Data',
#                   yaxis_title='Valor',
#                   autosize=True,
                  # showlegend=True,
#                  )

# fig.show()


In [ ]:
# dados['PETR4'].min(), dados['PETR4'].mean(), dados['PETR4'].max()

In [ ]:
# dados['Investimento'].min(), dados['Investimento'].mean(), dados['Investimento'].max()

In [ ]:
# dados.isnull().sum()

##Tratando os dados para usar na LSTM

In [ ]:
# PETR4
# !pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
valor_real = yf.download('PETR4.SA', start='2024-01-03', end='2024-02-16')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
valor_real.count()

Open         30
High         30
Low          30
Close        30
Adj Close    30
Volume       30
dtype: int64

In [ ]:
# dados.tail();

In [ ]:
valor_real.drop(columns=['Open','High','Low','Adj Close','Volume'], inplace=True)
# valor_real['Investimento Real']  = valor_real['Close'] * 300

In [ ]:
valor_real.reset_index(inplace=True)

In [ ]:
# valor_real

In [ ]:
valor_real[valor_real['Date'] == '2024-01-03']

,Date,Close
0,2024-01-03,38.959999


In [ ]:
valor_real['Investimento Real']  = valor_real['Close'] * 300

In [ ]:
# valor_real.to_excel('valor_real.xlsx', index=False)

In [ ]:
data = valor_real['Date']

In [ ]:
valor_real.drop(columns=['Date','Close'], inplace=True)
valor_real.head()

,Investimento Real
0,11687.999725
1,11589.000320
2,11616.000366
3,11529.000092
4,11429.999542


In [ ]:
# dados.isnull().sum()
valor_real.count()

Investimento Real    30
dtype: int64

In [ ]:
dados.drop(columns = 'Data', inplace = True)

In [ ]:
# colocando tudo na mesma escala
escalar = MinMaxScaler()
dados_norm = escalar.fit_transform(dados)

In [ ]:
valor_real_norm = escalar.fit_transform(valor_real)

In [ ]:
ultimos_30_dias = dados_norm[-30:, 5:]

In [ ]:
# prompt: selecione de dados_norm[-30:] a ultima coluna

ultimos_30_dias.shape


(30, 1)

In [ ]:
# Preparar os dados de entrada e saída
previsoes_dados = []
valor_real_dados = []

# Ajuste o loop para considerar previsões para 30 dias
for i in range(30, len(dados_norm) - 30):
    previsoes_dados.append(dados_norm[i-30:i, :])  # Usando todos os dados de 30 dias anteriores
    valor_real_dados.append(valor_real_norm)
    # valor_real_dados.append(ultimos_30_dias)

# Converter as listas em arrays NumPy
previsoes_dados, valor_real_dados = np.array(previsoes_dados), np.array(valor_real_dados)

# Verifique as formas dos dados preparados
print(f"Forma de previsoes_dados: {previsoes_dados.shape}")
print(f"Forma de valor_real_dados: {valor_real_dados.shape}")


Forma de previsoes_dados: (2425, 30, 6)
Forma de valor_real_dados: (2425, 30, 1)


In [ ]:
# ultimos_30_dias
valor_real_norm.shape, ultimos_30_dias.shape

((30, 1), (30, 1))

In [ ]:


ultimos_30_dias.shape

(30, 1)

In [ ]:

# Criar o modelo LSTM
lstm = Sequential()
lstm.add(Bidirectional(LSTM(units=100, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=(previsoes_dados.shape[1], previsoes_dados.shape[2]))))
lstm.add(Dropout(0.3))
lstm.add(Bidirectional(LSTM(units=50, return_sequences=True)))
lstm.add(Dropout(0.3))
lstm.add(Bidirectional(LSTM(units=50, return_sequences=True)))
lstm.add(Dropout(0.3))
lstm.add(LSTM(units=50))
lstm.add(Dropout(0.3))
lstm.add(Dense(units=30, activation='sigmoid'))  # Prevendo o valor contínuo de 'Investimento' sigmoid linear

lstm.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])


In [ ]:
previsoes_dados.shape[0], previsoes_dados.shape[1], previsoes_dados.shape[2]

(2425, 30, 6)

In [ ]:
# Treinar o modelo
lstm.fit(previsoes_dados, valor_real_dados, epochs=5, batch_size=32)

Epoch 1/5
76/76 [==============================] - 29s 131ms/step - loss: 0.0183 - mean_absolute_error: 0.0983
Epoch 2/5
76/76 [==============================] - 12s 154ms/step - loss: 0.0039 - mean_absolute_error: 0.0485
Epoch 3/5
76/76 [==============================] - 11s 150ms/step - loss: 0.0028 - mean_absolute_error: 0.0411
Epoch 4/5
76/76 [==============================] - 11s 151ms/step - loss: 0.0022 - mean_absolute_error: 0.0365
Epoch 5/5
76/76 [==============================] - 11s 142ms/step - loss: 0.0018 - mean_absolute_error: 0.0330


In [ ]:
# prompt: crie um gráfico co o erro da lstm

# plt.plot(lstm.history['loss'], label='train_loss')
# plt.plot(lstm.history['mean_absolute_error'], label='train_mae')
# plt.legend()
# plt.show()


In [ ]:

# dados_recentes
# dados[-30:].count()
d = dados[-30:]
d.count()

PETR4           30
Ibovespa        30
Brent           30
Selic           30
Dólar           30
Investimento    30
dtype: int64

In [ ]:
# dados.iloc[-30:, 6]
# dados_norm[-30:, 5]

In [ ]:
dados_recentes = dados_norm[-30:]

# Verificar a forma dos dados recentes
print(f"Forma de dados_recentes: {dados_recentes.shape}")

# Redimensionar os dados de entrada para a forma (1, 90, num_features)
dados_recentes = np.reshape(dados_recentes, (1, 30, dados_recentes.shape[1]))

# Fazer a previsão
previsao = lstm.predict(dados_recentes)

# Verificar a forma da previsão
print(f"Forma da previsão: {previsao.shape}")

# Exibir a previsão
# print("Previsão para os próximos 30 dias:", previsao)



Forma de dados_recentes: (30, 6)


1/1 [==============================] - 3s 3s/step
Forma da previsão: (1, 30)


In [ ]:
print("Previsão para os próximos 30 dias:", previsao)

Previsão para os próximos 30 dias: [[0.29113182 0.21816272 0.23846814 0.16632189 0.10356385 0.04466905
  0.09518416 0.12035969 0.21041328 0.10390362 0.06603603 0.04092877
  0.02529055 0.04031953 0.11568338 0.06115193 0.3640659  0.518715
  0.63801455 0.5902673  0.6868193  0.86334294 0.7577751  0.7957263
  0.8725061  0.9681835  0.9206241  0.80851054 0.73624104 0.9738179 ]]


In [ ]:
dados_preditos = escalar.inverse_transform(previsao)

In [ ]:
dados_preditos

array([[11675.609, 11571.19 , 11600.248, 11497.007, 11407.2  , 11322.921,
        11395.208, 11431.234, 11560.101, 11407.686, 11353.497, 11317.568,
        11295.19 , 11316.697, 11424.542, 11346.508, 11779.979, 12001.281,
        12171.999, 12103.673, 12241.838, 12494.443, 12343.377, 12397.684,
        12507.557, 12644.471, 12576.413, 12415.979, 12312.56 , 12652.533]],
      dtype=float32)

In [ ]:
dados_preditos = pd.DataFrame(dados_preditos)

In [ ]:
l = dados_preditos.values.flatten().tolist()
l = pd.DataFrame(l)

In [ ]:
modelo_lstm_30 = pd.concat([data, l], axis=1)
modelo_lstm_30.rename(columns = {0:'Predito', 'Date':'Data'}, inplace=True)

In [ ]:
valor_real.head()

,Investimento Real
0,11687.999725
1,11589.000320
2,11616.000366
3,11529.000092
4,11429.999542


In [ ]:
# modelo_lstm_30.to_excel('modelo_lstm_30.xlsx', index=False)

In [ ]:
mean_absolute_error(valor_real['Investimento Real'], modelo_lstm_30['Predito'])

18.639629618326822

In [ ]:
# # ultimos_30_dias,
# prev_30  = modelo_lstm_30.drop(columns='Data', inplace=True)
# scaler = MinMaxScaler()
# prev_30 = scaler.transform(prev_30)

# mean_absolute_error(ultimos_30_dias, prev_30)


In [ ]:
modelo_lstm_30.tail()

,Data,Predito
25,2024-02-07,12644.470703
26,2024-02-08,12576.413086
27,2024-02-09,12415.978516
28,2024-02-14,12312.559570
29,2024-02-15,12652.533203


In [ ]:
modelo_lstm_30.head()

,Data,Predito
0,2024-01-03,11675.609375
1,2024-01-04,11571.190430
2,2024-01-05,11600.248047
3,2024-01-08,11497.006836
4,2024-01-09,11407.200195


In [ ]:
# modelo_lstm_30.to_excel('modelo_lstm_30.xlsx', index=False)

In [ ]:
# valor_real.head()


In [ ]:
# valor_real

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(data=[go.Scatter(x=valor_real['Date'], y=valor_real['Investimento Real'], line=dict(color='black'))])
# fig.update_layout(title='Gráfico dos valores reais', xaxis_title='Data', yaxis_title='Valor')
# fig.show()


In [ ]:
# # Prepare data
# data = modelo_lstm_30['Data']
# predicted_values = modelo_lstm_30['Predito']
# real_values = valor_real['Investimento Real']

# # Create the figure
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=data, y=predicted_values, mode='lines', name='Valor Predito', line=dict(color='red')))
# fig.add_trace(go.Scatter(x=data, y=real_values, mode='lines', name='Valor Real', line=dict(color='black')))

# # Update layout
# fig.update_layout(
#     title='Previsão do Investimento utilizando LSTM',
#     xaxis_title='Data',
#     yaxis_title='Valor de Investimento',
# )

# # Show the figure
# fig.show()


In [ ]:
# x = dados[-30:]
# x .drop(columns='Data', inplace=True)
# scaler = MinMaxScaler()
# x = scaler.fit_transform(x)
